### The goal of this notebook is to scrap Google search results + parse them via Anthropic's Claude to get the date of municipal elections in each German state.

* It extracts the title and description from the results of the following Google Search: **"Kommunalwahlen [state] '[year]' 'am'"** for all combinations of state and election year. See for instance: [Kommunalwahlen Brandenburg '2014' 'am'](https://www.google.de/search?q=Kommunalwahlen+Brandenburg+%272014%27+%27am%27&sei=26cGaLHbBqirkdUPw_rquQ4).
  
* It asks [Anthropic's Claude](https://www.anthropic.com/claude) to parse the resulting output and to extract the date in YYYY-MM-DD format. As of 21/04/2025, using the API for Claude 3.7 Sonnet, it costs about $0.45. [Naturally, any other LLM could be used, but the author of the script got free credits from Anthropic :-)]

In [ ]:
# Install necessary libraries if not already installed ; ideally, we should have a requirements.txt file 
# But this script is very auxilliary to the main codebase, so leaving it as it is for now to keep it simple

# !pip install pandas googlesearch-python anthropic

In [ ]:
import os
import pandas as pd
from googlesearch import search
from pathlib import Path
import time

import anthropic

In [ ]:
PATH = str(Path.cwd().resolve().parents[2]) 

client = anthropic.Client(api_key=os.getenv("ANTHROPIC_API_KEY")) # Set your Anthropic API key as an environment variable
MODEL = "claude-3-7-sonnet-latest"

In [ ]:
def extract_election_date(text_snippet: str, state: str, year: int) -> str:
    """Uses an LLM to extract the municipal election date from text."""
    
    if not text_snippet or not client.api_key:
        print("Warning: No text snipppt or API key provided")
        return "NULL"

    prompt = f"""
    From the provided Google search result snippets relating to Kommunalwahlen (municipal elections) in the German state '{state}' for the year {year}, extract the main election date.

    Search Result Snippets:
    ---
    {text_snippet}
    ---

    Return only the single, most likely main election date. Format the date strictly as YYYY-MM-DD.
    Do not include run-off dates (Stichwahl) or publication dates of the snippets.
    If no specific date is found, return the exact string 'NULL'.
    """
    try:
        message = client.messages.create(
            model=MODEL,
            max_tokens=50,
            messages=[{"role": "user", "content": prompt}]
        )
        if message.content and isinstance(message.content, list) and message.content[0].text:
             result = message.content[0].text.strip()
             return result
        else:
            print(f"Warning: LLM returned unexpected content for {state} {year}")
            return "NULL"
    except Exception as e:
        print(f"Error calling LLM API for {state} {year}: {e}")
        return "NULL"

In [ ]:
def get_text_from_search(state: str, year: int, num_search_results=10) -> str:
    """
    Extract title + description from Google search results for a given state and year. 

    Args:
        state: The name of the German state.
        year: The election year.
        num_search_results: How many Google results to fetch.

    Returns:
        A single string containing titles and descriptions from search results,
        or an empty string if search fails or yields no results.
    """
    search_term = f"Kommunalwahlen {state} '{year}' 'am'" 
    print(f"Searching for: '{search_term}'")
    snippets = []
    try:
        search_generator = search(
            search_term,
            num_results=num_search_results,
            lang="de", 
            advanced=True, 
            sleep_interval=1 
        )

        for result in search_generator:
            title = result.title if result.title else ""
            desc = result.description if result.description else ""
            if title or desc: 
                 snippets.append(f"Title: {title}\nDescription: {desc}")

    except Exception as e:
        print(f"Error during search for {state} {year}: {e}")
        return "" 

    return "\n---\n".join(snippets)

In [ ]:
def main():

    # df_combinations contains all the possible combinations of state names x election years as outputted by `code/municipality_elections/01_municipal_unharm.R`.
    df_combinations = pd.read_csv(PATH + "/data/municipal_elections/processed/municipal_elections_combinations.csv")
    
    try:
        df_with_dates = pd.read_csv(PATH + "/data/municipal_elections/processed/municipal_elections_dates.csv")
    except FileNotFoundError:
        df_with_dates = None
    
    if df_with_dates:
        df_combinations = df_combinations[~df_combinations.set_index(['state', 'election_year']).index.isin(df_with_dates.set_index(['state', 'election_year']).index)]
        print(f"Filtered out {len(df_combinations)} combinations that already have dates.")

    n_to_process = len(df_combinations)

    n_processed = 0
    results = []

    for index, row in df_combinations.iterrows():
        state = row['state']
        year = row['election_year']

        search_content = get_text_from_search(state, year)

        if search_content:
            extracted_date = extract_election_date(search_content, state, year)
        else:
            print("  -> No search content found, skipping LLM.")
            extracted_date = "SEARCH_FAILED" 

        results.append({
            'state': state,
            'year': year,
            'extracted_date': extracted_date
        })

        time.sleep(1)
        n_processed += 1
        print(f"Processed {n_processed}/{n_to_process} combinations.") if n_processed % 10 == 0 else None        

    # convert results to DataFrame
    df_results = pd.DataFrame(results)
    if df_with_dates:
        df_results = pd.concat([df_with_dates, df_results], ignore_index=True)
        print(f"Combined with existing dates, total rows: {len(df_results)}")
    else: 
        print(f"New DataFrame created, total rows: {len(df_results)}")
        
    # save to CSV
    output_path = PATH + "/data/municipal_elections/processed/municipal_elections_dates.csv"
    df_results.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
dates = pd.read_csv(PATH + "/data/municipal_elections/processed/municipal_elections_dates.csv")
# Check for NULL or incorrect dates
dates.loc[(dates['extracted_date'] == 'NULL') | (dates['extracted_date'].str[:4] != dates['year'].astype(str))]